NLP Chatbot for Medical Personal Assistant:

Step 1: Setting up the Environment and Installing Necessary Libraries:

In [1]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install jupyter

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
pip install torch torchvision torchaudio

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
pip install flask

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
pip install tf-keras

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
pip install accelerate -U

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Step 2: Data Collection

In [12]:
import pandas as pd

data = {
    "Questions": [
        "What are the symptoms of flu?",
        "How do I treat a common cold?",
        "What is the recommended dosage for ibuprofen?",
        "What are the side effects of aspirin?",
        "How do I know if I have COVID-19?"
    ],
    "Answers": [
        "The symptoms of flu include fever, cough, sore throat, and muscle aches.",
        "Common cold can be treated with rest, hydration, and over-the-counter medications.",
        "The recommended dosage for ibuprofen is 200-400 mg every 4-6 hours.",
        "Side effects of aspirin include stomach pain, heartburn, and gastrointestinal bleeding.",
        "Symptoms of COVID-19 include fever, cough, shortness of breath, and loss of taste or smell."
    ]
}

df = pd.DataFrame(data)
df.to_csv('medical_data.csv', index=False)
df.head()


,Questions,Answers
0,What are the symptoms of flu?,"The symptoms of flu include fever, cough, sore..."
1,How do I treat a common cold?,"Common cold can be treated with rest, hydratio..."
2,What is the recommended dosage for ibuprofen?,The recommended dosage for ibuprofen is 200-40...
3,What are the side effects of aspirin?,"Side effects of aspirin include stomach pain, ..."
4,How do I know if I have COVID-19?,"Symptoms of COVID-19 include fever, cough, sho..."


Step 3: Preprocessing

In [13]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token.isalnum() and token not in stop_words]
    return ' '.join(filtered_tokens)

df['Processed_Questions'] = df['Questions'].apply(preprocess_text)
df['Processed_Answers'] = df['Answers'].apply(preprocess_text)
df.head()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lenocvo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lenocvo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Questions,Answers,Processed_Questions,Processed_Answers
0,What are the symptoms of flu?,"The symptoms of flu include fever, cough, sore...",symptoms flu,symptoms flu include fever cough sore throat m...
1,How do I treat a common cold?,"Common cold can be treated with rest, hydratio...",treat common cold,common cold treated rest hydration medications
2,What is the recommended dosage for ibuprofen?,The recommended dosage for ibuprofen is 200-40...,recommended dosage ibuprofen,recommended dosage ibuprofen mg every hours
3,What are the side effects of aspirin?,"Side effects of aspirin include stomach pain, ...",side effects aspirin,side effects aspirin include stomach pain hear...
4,How do I know if I have COVID-19?,"Symptoms of COVID-19 include fever, cough, sho...",know,symptoms include fever cough shortness breath ...


Step 4: Loading Pre-trained Model:

In [14]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

# Verify model loading
print("Model loaded successfully!")

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded successfully!


Step 5: Loading and Preparing the SQuAD Dataset:

In [16]:
from datasets import load_dataset

# Load the SQuAD dataset
squad = load_dataset('squad')

def preprocess_function(examples):
    questions = [q.strip() for q in examples['question']]
    inputs = tokenizer(
        questions,
        examples['context'],
        max_length=384,
        truncation='only_second',
        padding='max_length',
        return_tensors='pt'
    )
    # start_positions = []
    # end_positions = []
    # for i in range(len(examples['answers'])):
        # start = examples['context'][i].find(examples['answers'][i]['text'][0])
        # end = start + len(examples['answers'][i]['text'][0])
        # start_positions.append(start)
        # end_positions.append(end)

    start_positions = [context.index(ans['text'][0]) for context, ans in zip(examples['context'], examples['answers'])]
    end_positions = [start + len(ans['text'][0]) for start, ans in zip(start_positions, examples['answers'])]
    inputs['start_positions'] = start_positions
    inputs['end_positions'] = end_positions
    return inputs

tokenized_squad = squad.map(preprocess_function, batched=True)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

Step 6: Fine-tuning the Model:

In [18]:
import tensorflow as tf
import logging

# Suppress TensorFlow warnings
tf.get_logger().setLevel(logging.ERROR)

# import tensorflow as tf

# # Replace deprecated call
# loss = tf.compat.v1.losses.sparse_softmax_cross_entropy(labels, predictions)

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad['train'],
    eval_dataset=tokenized_squad['validation']
)

trainer.train()

  0%|          | 0/32850 [00:00<?, ?it/s]

KeyboardInterrupt: 

Step 7: Saving the Fine-tuned Model:

In [ ]:
model.save_pretrained('fine-tuned-bert-question-answering')
tokenizer.save_pretrained('fine-tuned-bert-question-answering')

Step 8: Setting the Inference Function/Using the Model for Predictions:

In [25]:
def answer_question(question, context):
    inputs = tokenizer.encode_plus(question, context, return_tensors='pt')
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    outputs = model(input_ids, attention_mask=attention_mask)
    start_scores, end_scores = outputs.start_logits, outputs.end_logits

    all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    answer = ' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1])
    return answer.replace(' ##', '')

# Test the function
question = "What are the symptoms of flu?"
context = "The symptoms of flu include fever, cough, sore throat, and muscle aches."
print(answer_question(question, context))
# print(answer_question(context))

[SEP] the symptoms of flu include fever , cough , sore


Step 9: Calling an API endpoint:

In [26]:
import requests

# Define the endpoint URL
url = 'http://127.0.0.1:5000/api_endpoint'  # Replace with your actual endpoint

# Define the data to send to the endpoint
data = {
    'question': 'What are the symptoms of flu?',
    'context': 'The symptoms of flu include fever, cough, sore throat, and muscle aches.'
}

# Make a POST request to the Flask app
response = requests.post(url, json=data)

# Print the response from the Flask app
# print(response.json())
print(response.content)

# Try to parse the response as JSON
try:
    response_json = response.json()
    print(response_json)
except requests.exceptions.JSONDecodeError:
    print("Response is not in JSON format.")

b'<!doctype html>\n<html lang=en>\n  <head>\n    <title>NameError: name &#39;conte&#39; is not defined\n // Werkzeug Debugger</title>\n    <link rel="stylesheet" href="?__debugger__=yes&amp;cmd=resource&amp;f=style.css">\n    <link rel="shortcut icon"\n        href="?__debugger__=yes&amp;cmd=resource&amp;f=console.png">\n    <script src="?__debugger__=yes&amp;cmd=resource&amp;f=debugger.js"></script>\n    <script>\n      var CONSOLE_MODE = false,\n          EVALEX = true,\n          EVALEX_TRUSTED = false,\n          SECRET = "wH57maM9vMB1mplPObdY";\n    </script>\n  </head>\n  <body style="background-color: #fff">\n    <div class="debugger">\n<h1>NameError</h1>\n<div class="detail">\n  <p class="errormsg">NameError: name &#39;conte&#39; is not defined\n</p>\n</div>\n<h2 class="traceback">Traceback <em>(most recent call last)</em></h2>\n<div class="traceback">\n  <h3></h3>\n  <ul><li><div class="frame" id="frame-1864734476752">\n  <h4>File <cite class="filename">"C:\\Users\\lenocvo\\Do